**Simulation Design for Mitochondrial Lineage Tracing and Single-Cell Transcriptomic Data**

### **1. Overview**

The simulation consists of four main components:

1. **Stem Cell Growth Simulation**: Modeling the proliferation of stem cells using the Gillespie algorithm.
2. **Mitochondrial Genome Proliferation**: Simulating mtDNA replication, mutation accumulation, and segregation during cell division.
3. **Cell Differentiation Simulation**: Modeling asymmetric division of progenitor cells and differentiation into various cell types.
4. **Single-Cell Transcriptomic Data Simulation**: Generating gene expression data using latent variables and sampling from a Zero-Inflated Negative Binomial (ZINB) distribution.

---

### **2. Simulation of Stem Cell Growth**

**2.1. Model Description**

We simulate stem cell growth by conceptualizing cell division as a continuous-time Markov process. The division rate \( p(t) \) of a stem cell at time \( t \) is defined as:

\[
p(t) = r \left(1 - \frac{1}{1 + e^{-\kappa (t - t_0)}}\right),
\]

where:

- \( r \) is the maximum division rate.
- \( \kappa \) controls the rate at which the growth accelerates.
- \( t_0 \) represents the inflection point in time when the growth rate changes significantly.

**2.2. Simulation Implementation**

We use the Gillespie stochastic simulation algorithm to model the timing of cell divisions. Starting from a single stem cell, we simulate cell divisions until the desired population size is reached.

---

### **3. Simulation of Mitochondrial Genome Proliferation**

**3.1. Initial mtDNA Population**

Each stem cell contains an initial population of mtDNA molecules. We simulate the replication and death of mtDNA molecules within the cell until a target copy number (e.g., 500 copies) is reached.

**3.2. mtDNA Replication and Mutation**

During cell division, mtDNA molecules replicate, and mutations can occur. The key aspects of the mtDNA dynamics include:

- **Replication**: Each mtDNA molecule replicates once.
- **Mutation Accumulation**: Mutations occur during replication, with the number of mutations following a Poisson distribution.
- **Segregation**: mtDNA molecules are randomly segregated into daughter cells during cell division.

**3.3. Mutation Rate Estimation**

Assuming a per-mitosis mutation rate for mtDNA that is higher than nuclear DNA, we estimate the expected number of mtDNA mutations per cell division:

\[
\text{Expected mutations per division} = \mu \times \text{mtDNA copies},
\]

where \( \mu \) is the mutation rate per mtDNA molecule per division.

---

### **4. Simulation of Cell Differentiation**

**4.1. Differentiation Models**

After stem cell growth, cells differentiate according to specified models:

- **Linear Differentiation**: Stem cells differentiate into progenitor cells, which further differentiate into a single cell type.
- **Bifurcated Differentiation**: Progenitor cells differentiate into multiple cell types.

**4.2. Asymmetric Division of Progenitor Cells**

Progenitor cells undergo asymmetric division:

- One daughter cell remains a progenitor cell capable of self-renewal and further differentiation.
- The other daughter cell differentiates into a specialized cell type and no longer divides.

**4.3. Differentiation Probability**

Cell differentiation is modeled as a continuous-time Markov process, with a differentiation probability \( q_{ij}(t) \) at time \( t \):

\[
q_{ij}(t) = 1 - p_i(t),
\]

where \( p_i(t) \) is the probability that a cell remains in its current state.

---

### **5. Simulation of Single-Cell Transcriptomic Data**

**5.1. Gene Expression Model**

We simulate gene expression for each cell, accounting for cell-type-specific and shared genes:

- **Cell-Type-Specific Genes**: Comprise 40% of the simulated genes, following distinct expression programs in different cell types.
- **Shared Genes**: The remaining genes, with consistent expression across cell types.

**5.2. Two-Step Expression Simulation**

1. **Latent Expression Generation**:

   For each gene, we generate a latent expression variable \( z_t \) for cell generation \( t \):

   \[
   z_t \sim \text{Normal}(\mu_t, \sigma^2),
   \]

   where \( \mu_t \) is the mean expression level dependent on the cell's generation and cell type.

2. **Sampling Expression Data**:

   The observed gene expression \( x_t \) is sampled from a Zero-Inflated Negative Binomial distribution:

   \[
   x_t \sim \text{ZINB}(\lambda = e^{z_t}, \theta, \pi),
   \]

   where:

   - \( \lambda \) is the mean expression level.
   - \( \theta \) is the dispersion parameter.
   - \( \pi \) is the zero-inflation probability.

**5.3. Parameters**

- **Base Expression Levels**: Randomly assigned for each gene.
- **Expression Rates**: Define how gene expression changes with cell generation.
- **Dispersion and Zero-Inflation Parameters**: Set to mimic realistic single-cell RNA-seq data.

---

### **6. Simulation of Sequencing Read Depth for Mitochondrial Mutations**

**6.1. Motivation**

In sequencing data, the total read depth (DP) represents the number of sequencing reads covering a particular genomic position, while the allele depth (AD) represents the number of reads supporting a specific allele (e.g., a mutation) at that position. Including DP and AD in our simulation allows us to model the stochastic nature of sequencing and the variability in detecting mitochondrial mutations.

**6.2. Modeling Read Depth**

For each mitochondrial mutation in each cell:

- **Total Read Depth (DP)**: Simulated using a suitable distribution (e.g., Poisson or Negative Binomial) to reflect variability in sequencing coverage.
- **Allele Read Depth (AD)**: Simulated based on the heteroplasmy level of the mutation and the total read depth.

**6.3. Implementation Details**

- **Per-Mutation DP Simulation**: We simulate the DP for each mutation site in each cell.
- **AD Calculation**: Given the DP and the heteroplasmy level (proportion of mtDNA molecules carrying the mutation), we simulate the AD using a binomial distribution.
- **Error Modeling**: We can include sequencing errors by introducing a base error rate, affecting the observed AD.

---

### **7. Python Code Implementation**

Below is the complete Python code implementing the simulation design. Explanations are provided within the code comments.

In [1]:
import numpy as np
import pandas as pd
import random
from collections import deque

# Set random seed for reproducibility
np.random.seed(42)

# --- Simulation Parameters ---

# Stem cell growth parameters
r = 1.0        # Maximum division rate
kappa = 0.1    # Growth acceleration rate
t0 = 50        # Inflection point in time

# mtDNA parameters
mtDNA_initial_count = 500
mtDNA_mutation_rate_per_mitosis = 0.4   # Mutations per cell division

# Sequencing read depth parameters
mean_dp = 100     # Mean total read depth per mutation site
dp_dispersion = 0.2  # Dispersion parameter for total read depth (overdispersion)
base_error_rate = 0.001  # Sequencing error rate

# Cell differentiation parameters
kappa_diff = 0.1  # Differentiation rate parameter
t0_diff = 50      # Differentiation inflection time

# Gene expression parameters
num_genes = 4000
cell_type_specific_genes = int(num_genes * 0.4)
shared_genes = num_genes - cell_type_specific_genes
alpha = 0.1  # Dispersion parameter for Negative Binomial
zero_inflation_prob = 0.1  # Probability of zero-inflation in gene expression

# Simulation settings
total_cells = 1000  # Total number of stem cells to simulate
max_generations = 10  # Max generations for stem cell divisions

# --- Class Definitions ---

class mtDNA:
    def __init__(self, id, mutations=None):
        self.id = id
        self.mutations = set(mutations) if mutations else set()

class Cell:
    def __init__(self, id, parent_id, generation, time, mtDNA_list=None, cell_type='StemCell'):
        self.id = id
        self.parent_id = parent_id
        self.generation = generation
        self.time = time
        self.mtDNA_list = mtDNA_list if mtDNA_list else []
        self.cell_type = cell_type
        self.children = []
        self.mutation_profile = {}  # To store DP and AD for mutations

# --- Functions for mtDNA Dynamics ---

def introduce_mutations(mtDNA_list, mutation_rate):
    """Introduce mutations into mtDNA molecules."""
    for mt in mtDNA_list:
        num_mutations = np.random.poisson(mutation_rate)
        for _ in range(num_mutations):
            mutation = f"m{np.random.randint(1e6)}"
            mt.mutations.add(mutation)
    return mtDNA_list

def replicate_and_segregate_mtDNA(mtDNA_list):
    """Replicate and segregate mtDNA during cell division."""
    # Replicate mtDNA
    replicated_mtDNA = []
    for mt in mtDNA_list:
        replicated_mtDNA.append(mt)
        replicated_mtDNA.append(mtDNA(mt.id, mt.mutations.copy()))
    # Introduce mutations
    mutation_rate = mtDNA_mutation_rate_per_mitosis / len(replicated_mtDNA)
    replicated_mtDNA = introduce_mutations(replicated_mtDNA, mutation_rate)
    # Segregate mtDNA to daughter cells
    total_mtDNA = len(replicated_mtDNA)
    segregation_counts = np.random.multinomial(total_mtDNA, [0.5, 0.5])
    indices = np.random.permutation(total_mtDNA)
    daughter1_indices = indices[:segregation_counts[0]]
    daughter2_indices = indices[segregation_counts[0]:]
    daughter1_mtDNA = [replicated_mtDNA[i] for i in daughter1_indices]
    daughter2_mtDNA = [replicated_mtDNA[i] for i in daughter2_indices]
    return [daughter1_mtDNA, daughter2_mtDNA]

# --- Functions for Cell Growth and Differentiation ---

def division_rate(t, r, kappa, t0):
    """Calculate cell division rate at time t."""
    return r * (1 - 1 / (1 + np.exp(-kappa * (t - t0))))

def differentiation_probability(t, kappa_diff, t0_diff):
    """Calculate cell differentiation probability at time t."""
    return 1 - (1 / (1 + np.exp(-kappa_diff * (t - t0_diff))))

def simulate_stem_cell_growth():
    """Simulate stem cell growth using the Gillespie algorithm."""
    cells = []
    cell_id_counter = 0
    time = 0.0
    cell_queue = deque()

    # Initialize the first cell
    mtDNA_population = [mtDNA(i) for i in range(mtDNA_initial_count)]
    initial_cell = Cell(cell_id_counter, None, 0, time, mtDNA_population.copy(), 'StemCell')
    cells.append(initial_cell)
    cell_queue.append(initial_cell)
    cell_id_counter += 1

    # Simulate stem cell divisions
    while len(cells) < total_cells:
        current_cell = cell_queue.popleft()
        # Calculate division time
        div_rate = division_rate(current_cell.time, r, kappa, t0)
        division_t = np.random.exponential(scale=1.0 / div_rate)
        time += division_t

        # Replicate and segregate mtDNA
        daughter_mtDNAs = replicate_and_segregate_mtDNA(current_cell.mtDNA_list)

        # Create daughter cells
        for i in range(2):
            new_cell = Cell(
                cell_id_counter,
                current_cell.id,
                current_cell.generation + 1,
                time,
                daughter_mtDNAs[i],
                'StemCell'
            )
            cells.append(new_cell)
            cell_queue.append(new_cell)
            current_cell.children.append(new_cell.id)
            cell_id_counter += 1
    return cells

def simulate_cell_differentiation(cells):
    """Simulate cell differentiation after stem cell growth."""
    differentiated_cells = []
    progenitor_cells = []
    for cell in cells:
        if cell.cell_type == 'StemCell':
            # Decide whether the stem cell differentiates into a progenitor cell
            diff_prob = differentiation_probability(cell.time, kappa_diff, t0_diff)
            if np.random.rand() < diff_prob:
                cell.cell_type = 'Progenitor'
                progenitor_cells.append(cell)
        elif cell.cell_type == 'Progenitor':
            # Progenitor cells divide asymmetrically
            daughter_cells = progenitor_asymmetric_division(cell)
            differentiated_cells.extend(daughter_cells)
    return cells + differentiated_cells

def progenitor_asymmetric_division(cell):
    """Simulate asymmetric division of progenitor cells."""
    # Replicate and segregate mtDNA
    daughter_mtDNAs = replicate_and_segregate_mtDNA(cell.mtDNA_list)

    # One daughter remains progenitor
    progenitor_daughter = Cell(
        cell.id * 2,
        cell.id,
        cell.generation + 1,
        cell.time + 1,
        daughter_mtDNAs[0],
        'Progenitor'
    )

    # The other daughter differentiates
    differentiated_daughter = Cell(
        cell.id * 2 + 1,
        cell.id,
        cell.generation + 1,
        cell.time + 1,
        daughter_mtDNAs[1],
        'DifferentiatedType1'  # Assuming linear differentiation
    )

    return [progenitor_daughter, differentiated_daughter]

# --- Functions for Sequencing Read Depth Simulation ---

def simulate_read_depth(cell, all_mutations):
    """Simulate DP and AD for mitochondrial mutations in a cell."""
    mutation_index = {mutation: idx for idx, mutation in enumerate(all_mutations)}
    num_mutations = len(all_mutations)
    mtDNA_count = len(cell.mtDNA_list)

    # Calculate heteroplasmy levels
    mutation_counts = {}
    for mt in cell.mtDNA_list:
        for mutation in mt.mutations:
            mutation_counts[mutation] = mutation_counts.get(mutation, 0) + 1

    heteroplasmy = {mutation: count / mtDNA_count for mutation, count in mutation_counts.items()}

    # Simulate total read depth (DP) for each mutation site
    dp_values = np.random.negative_binomial(
        n=1 / dp_dispersion,
        p=1 / (1 + mean_dp * dp_dispersion),
        size=num_mutations
    )

    dp_values = dp_values + mean_dp  # Adjusting to have mean around mean_dp

    # Simulate allele depth (AD) based on heteroplasmy and DP
    ad_values = []
    dp_list = []
    for idx, mutation in enumerate(all_mutations):
        dp = max(int(dp_values[idx]), 1)  # Ensure at least 1 read
        dp_list.append(dp)
        h = heteroplasmy.get(mutation, base_error_rate)  # Use error rate if mutation not present
        # Include sequencing errors
        effective_h = h * (1 - base_error_rate) + (1 - h) * base_error_rate
        ad = np.random.binomial(dp, effective_h)
        ad_values.append(ad)

    # Store DP and AD in cell's mutation_profile
    cell.mutation_profile = {
        mutation: {'DP': dp_list[idx], 'AD': ad_values[idx]}
        for idx, mutation in enumerate(all_mutations)
    }

# --- Functions for Gene Expression Simulation ---

def generate_gene_params():
    """Generate parameters for gene expression simulation."""
    gene_ids = [f'Gene_{i}' for i in range(num_genes)]
    specific_gene_ids = random.sample(gene_ids, cell_type_specific_genes)
    shared_gene_ids = list(set(gene_ids) - set(specific_gene_ids))

    gene_params = {}
    for gene_id in gene_ids:
        if gene_id in specific_gene_ids:
            gene_params[gene_id] = {
                'base_expression': np.random.uniform(1, 5),
                'expression_rate': np.random.uniform(0.1, 0.5),
                'sigma': 1.0
            }
        else:
            gene_params[gene_id] = {
                'base_expression': np.random.uniform(1, 5),
                'expression_rate': 0.0,
                'sigma': 1.0
            }
    return gene_params

def generate_latent_expression(cell, gene_params):
    """Generate latent expression values for a cell."""
    z_t = {}
    for gene_id, params in gene_params.items():
        loc = params['base_expression'] + params['expression_rate'] * cell.generation
        scale = params['sigma']
        z_t[gene_id] = np.random.normal(loc=loc, scale=scale)
    return z_t

def sample_expression(z_t, alpha, zero_inflation_prob):
    """Sample gene expression counts from a ZINB distribution."""
    x_t = {}
    for gene_id, z in z_t.items():
        mu = np.exp(z)
        r = 1 / alpha
        p = r / (r + mu)
        if np.random.rand() < zero_inflation_prob:
            x_t[gene_id] = 0
        else:
            x_t[gene_id] = np.random.negative_binomial(n=r, p=p)
    return x_t

def simulate_gene_expression_for_cells(cells, gene_params):
    """Simulate gene expression for a list of cells."""
    expression_data = {}
    for cell in cells:
        z_t = generate_latent_expression(cell, gene_params)
        x_t = sample_expression(z_t, alpha, zero_inflation_prob)
        expression_data[cell.id] = x_t
    expression_df = pd.DataFrame.from_dict(expression_data, orient='index')
    return expression_df

# --- Main Simulation Execution ---

# Step 1: Simulate stem cell growth
stem_cells = simulate_stem_cell_growth()

# Step 2: Simulate cell differentiation
all_cells = simulate_cell_differentiation(stem_cells)

# Step 3: Collect all unique mutations across cells
all_mutations = set()
for cell in all_cells:
    for mt in cell.mtDNA_list:
        all_mutations.update(mt.mutations)
all_mutations = sorted(all_mutations)

# Step 4: Simulate sequencing read depth (DP and AD) for mitochondrial mutations
for cell in all_cells:
    simulate_read_depth(cell, all_mutations)

# Step 5: Generate gene expression parameters
gene_params = generate_gene_params()

# Step 6: Simulate gene expression
expression_df = simulate_gene_expression_for_cells(all_cells, gene_params)

# --- Data Output and Visualization ---

# Example: Display DP and AD for the first 5 cells
print("Mitochondrial Mutation Profiles (first 5 cells):")
for cell in all_cells[:5]:
    print(f"Cell ID: {cell.id}, Mutation Profile:")
    for mutation, counts in cell.mutation_profile.items():
        print(f"  Mutation: {mutation}, DP: {counts['DP']}, AD: {counts['AD']}")
    print()

# Example: Display gene expression data for the first 5 cells
print("Simulated Gene Expression Data (first 5 cells):")
print(expression_df.head())

/tmp/ipykernel_31096/1247361599.py:112: RuntimeWarning: divide by zero encountered in scalar divide
  division_t = np.random.exponential(scale=1.0 / div_rate)


Simulated Gene Expression Data (first 5 cells):
   Gene_0  Gene_1  Gene_2  Gene_3  Gene_4  Gene_5  Gene_6  Gene_7  Gene_8  \
0      69      11     160      29      35       0       3      19      79   
1       0      14      41      60       0       0      59     131    1149   
2     495       9       7      54      25       2       1      75      43   
3     133       0     323       0      27       1       0      35      20   
4     882      15     161     275      67       0       5     271      59   

   Gene_9  ...  Gene_3990  Gene_3991  Gene_3992  Gene_3993  Gene_3994  \
0      11  ...         44          6          5         10         22   
1       6  ...          0          3         52         12         34   
2       9  ...         43          1          0         89         12   
3      19  ...         22          2          2          1         52   
4      28  ...         20         27         10         39        118   

   Gene_3995  Gene_3996  Gene_3997  Gene_3998  Gen

### **8. Code Explanation**

- **Class Definitions**:
  - `mtDNA` class represents individual mtDNA molecules, tracking their mutations.
  - `Cell` class represents cells in the simulation, storing lineage information and cell type.

- **mtDNA Dynamics Functions**:
  - `introduce_mutations` introduces mutations into mtDNA molecules during replication.
  - `replicate_and_segregate_mtDNA` handles mtDNA replication and random segregation into daughter cells.

- **Cell Growth and Differentiation Functions**:
  - `simulate_stem_cell_growth` models stem cell divisions using the Gillespie algorithm.
  - `simulate_cell_differentiation` models differentiation of stem cells into progenitor cells and further into differentiated cells.
  - `progenitor_asymmetric_division` simulates asymmetric division of progenitor cells.

- **Gene Expression Simulation Functions**:
  - `generate_gene_params` initializes gene-specific parameters for the simulation.
  - `generate_latent_expression` computes the latent expression levels \( z_t \) for genes in a cell.
  - `sample_expression` samples gene expression counts from a ZINB distribution.
  - `simulate_gene_expression_for_cells` simulates gene expression data for all cells.

- **Main Execution**:
  - The simulation proceeds through stem cell growth, differentiation, parameter generation, and gene expression simulation.
  - The final gene expression data is stored in `expression_df`.

#### **Simulating Read Depth (DP and AD)**

- **Function `simulate_read_depth`**:
  - For each cell, we calculate the heteroplasmy levels of mutations.
  - Total read depth (DP) for each mutation site is simulated using a Negative Binomial distribution to model overdispersion in sequencing coverage.
  - Allele depth (AD) is simulated using a Binomial distribution based on the heteroplasmy level and the total read depth.
  - Sequencing errors are included by adjusting the effective heteroplasmy with the base error rate.

#### **Parameters Used**

- **`mean_dp`**: The average total read depth per mutation site. This value can be adjusted to reflect different sequencing depths.
- **`dp_dispersion`**: Controls the variance of the total read depth. A higher value indicates more overdispersion.
- **`base_error_rate`**: The probability of a sequencing error at any given base. This affects the observed allele depth, particularly for low-frequency mutations.

#### **Storing DP and AD**

- Each cell's mitochondrial mutation profile is stored in the `mutation_profile` attribute, which is a dictionary mapping mutations to their DP and AD values.

---

### **9. Results and Interpretation**

- **Gene Expression Data**:
  - The simulated gene expression data (`expression_df`) contains realistic counts that mimic single-cell RNA-seq data.
  - The data reflects cell-type-specific expression patterns due to the inclusion of cell-type-specific genes.

- **Cell Type Distribution**:
  - The simulation tracks the number of cells in each cell type, allowing for analysis of differentiation dynamics.